In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import datetime
import os
import matplotlib.pyplot as plt

# Process to one spreadsheet

In [20]:
dfs = dict()
path = 'data/projectData/'
for file in os.listdir(path):
    if file[0] != '.':
        df = pd.read_excel(f'{path}{file}', index_col=0)
        df['dateSeen'] = pd.to_datetime(df['dateSeen'], unit='s', utc=True).dt.tz_localize(None)
        print(file, len(df))
        dfs[file] = df

apnews.com_07-17-2022_trial.xlsx 1446
sputniknews.com_07-14-2022_trial.xlsx 1407
reuters.com_07-15-2022_trial.xlsx 489
bbc.co.uk_07-15-2022_trial.xlsx 743
tass.com_07-14-2022_trial.xlsx 1952


In [27]:
westernDfs = pd.concat([dfs['apnews.com_07-17-2022_trial.xlsx'],dfs['reuters.com_07-15-2022_trial.xlsx'],dfs['bbc.co.uk_07-15-2022_trial.xlsx']])
print('initial', len(westernDfs))
westernDfs = westernDfs[westernDfs['text'].apply(lambda x: isinstance(x, str))]
print('type text string', len(westernDfs))
westernDfs.tail()

initial 2678
type text string 2667


,newsOutlet,dateSeen,url,title,language,sourceCountry,text
738,bbc.co.uk,2022-03-12 08:00:00,https://www.bbc.co.uk/news/uk-england-derbyshi...,Ukraine families : Mum escapes to join daughte...,English,United Kingdom,A woman who helped her mum escape from the war...
739,bbc.co.uk,2022-03-11 08:00:00,https://www.bbc.co.uk/news/uk-england-beds-buc...,Ukrainian mother freed by St Albans judge to t...,English,United Kingdom,A mother convicted of abducting her daughter h...
740,bbc.co.uk,2022-03-11 08:00:00,https://www.bbc.co.uk/news/uk-wales-60705619,Ukraine crisis : Mark Drakeford urges visa - f...,English,United Kingdom,Ukrainian refugees should be allowed into the ...
741,bbc.co.uk,2022-03-01 08:00:00,https://www.bbc.co.uk/news/world-europe-60567162,"Ukraine : Kharkiv attack was a war crime , say...",English,United Kingdom,Building hit by missile in Kharkiv's Freedom S...
742,bbc.co.uk,2022-03-07 08:00:00,https://www.bbc.co.uk/news/uk-england-coventry...,Ukrainian doctor in Coventry urges more visa h...,English,United Kingdom,A doctor is urging the government to make it e...


In [28]:
russianDfs = pd.concat([dfs['sputniknews.com_07-14-2022_trial.xlsx'],dfs['tass.com_07-14-2022_trial.xlsx']])
print('all types', len(russianDfs))
russianDfs = russianDfs[russianDfs['text'].apply(lambda x: isinstance(x, str))]
print('type text string', len(russianDfs))
russianDfs.tail()

all types 3359
type text string 3344


,newsOutlet,dateSeen,url,title,language,sourceCountry,text
1947,tass.com,2022-03-01 08:00:00,https://tass.com/politics/1414129,Kremlin spokesman says has no data about Russi...,English,NaN,"MOSCOW, March 1. /TASS/. Russian presidential ..."
1948,tass.com,2022-03-01 08:00:00,https://tass.com/politics/1414255,West will adopt new sanctions packages against...,English,NaN,"BERLIN, March 1. /TASS/. German Chancellor Ola..."
1949,tass.com,2022-03-01 08:00:00,https://tass.com/world/1414101,Aggressive actions against Russia become ultra...,English,NaN,"MOSCOW, March 1. /TASS/. Russia has a history ..."
1950,tass.com,2022-03-02 08:00:00,https://tass.com/economy/1414631,G7 countries ready to exert more financial pre...,English,NaN,"WASHINGTON, March 2. /TASS/. The United States..."
1951,tass.com,2022-03-01 08:00:00,https://tass.com/economy/1414507,UK imposes sanctions against Russian Direct In...,English,NaN,"LONDON, March 1. /TASS/. The UK has imposed sa..."


In [29]:
result = pd.concat([westernDfs.sample(n=2500), russianDfs.sample(n=2500)])
print(len(result))
result.head()

5000


,newsOutlet,dateSeen,url,title,language,sourceCountry,text
214,reuters.com,2021-11-12 08:00:00,https://www.reuters.com/article/worldNews/idUS...,Neighbours of Belarus say migrant crisis risks...,English,United States,"KAPCIAMIESTIS, Lithuania/WARSAW Nov 11 (Reuter..."
655,bbc.co.uk,2022-03-09 08:00:00,https://www.bbc.co.uk/news/uk-60679658,Ukraine - Russia war : united kingdom agrees t...,English,United Kingdom,The UK is increasing its supply of weapons to ...
246,reuters.com,2021-11-07 07:00:00,https://www.reuters.com/business/healthcare-ph...,Global COVID - 19 cases near 250 million as De...,English,United States,Medical workers wearing personal protective eq...
663,apnews.com,2022-01-29 08:00:00,https://apnews.com/article/russia-ukraine-euro...,AP PHOTOS : A Ukrainian village awaits the nex...,English,United States,"ZOLOTE, Ukraine (AP) — In the Ukrainian villag..."
1328,apnews.com,2022-03-12 08:00:00,https://apnews.com/a5a723076b335ec36452bb27ddf...,That what dads do : US man goes to Ukraine f...,English,United States,"FITCHBURG, Mass. (AP) — As millions of refugee..."


In [30]:
result.to_excel('apnews_bbc_reuters-sputnik_tass_5000_articles.xlsx',index=True)